# Overview of .ipynb notebooks and their associated .jl files

The notebooks in this directory contain code to run and differentiate ProAnti models. Each notebook contains some core source code, that gets auto-extracted into a Julia .jl file, plus documentation and examples that are for illustration and are not extracted into the .jl file.

The notebooks are:

**Scraper.ipynb** : defines and describes how to auto-scrape notebooks into .jl files

**General Utilities.ipynb**  : various utilities, extracted into <span style="color:red">general_utils.jl</span>.

**Constrained Parabolic Minimization.ipynb** : finding the minimum of a parabolic surface, constrained to lie within a given radius from the origin. Used by the next notebook, and extracted into <span style="color:red">constrained_parabolic_minimization.jl</span>

**Cost Function Minimization and Hessian Utilities.ipynb** : like the title says, for minimizing functions and finding their gradients and hessians, extracted into <span style="color:red">hessian_utils.jl</span>

**Rate Networks.ipynb** : running and differentiating arbitrary rate-based neural networks, extracted into <span style="color:red">rate_networks.jl</span>. Loading `rate_networks.jl` will also load all three of `general_utils.jl`, `constrained_parabolic_minimization.jl`, and `hessian_utils.jl`.

**ProAnti.ipynb** : Uses all of the above to run and optimize ProAnti models. The model definition, trial running, and the cost function we started with are extracted into <span style="color:red">pro_anti.jl</span>. An example of code for running an optimization farm is extracted into <span style="color:red">farming.jl</span>

# Running on the Google compute cloud

You'll want to both install [google compute command line tools](https://cloud.google.com/sdk/) and find your way to the [compute Console web site](https://console.cloud.google.com/compute/instances?project=autoreminder-1219).  Our project is called "Autoreminder".

Going from the Console web site, you can go to the "VM Instances" and there see the VMs we have configured so far. Their disk drive is persistent, meaning that whatever is on it stays on it even when the machine is turned off. We get charged a very small amount per GB of hard disk, regardless of whether VMs are on or off. This amount is so small that I don't think about it. For CPU cycles, we get charged by the core by the minute when a VM is turned on.  A 64-core VM that is turned on runs at about $3/hour (again, regardless of CPU usage).

Our quota is up to 2048 CPUs, located in Google's "us-east1-c" zone.

## Making a new instance and initializing it (i.e., adding julia, git, etc.) 

### Making the instance

To make a new instance, do it from the Console web site. Choose the "us-east1-c" zone, which is where we have a high CPU quota, and choose the default Debian 9 Linux operating system. Most of the machines made so far are 64-core machines with 10 GB hard drive (we don't need a lot of disk space).

### Logging into the instance

Lets say you made an instance (i.e., a VM) and called it "my_instance". If you have installed the gcloud command-line tool, you can ssh into your new instance using

    gcloud compute ssh my_google_username@my_instance
 
The gcloud tool takes care of figuring out the correct IP address, etc. And I _believe_ that the first time you try this it'll guide you through setting up the ssh keys, but I'm not totally sure about that. Further googling on how to intialize your ssh keys might be needed to allow you to log in without having to type a password every time.

You can ssh in and then treat the VM as a regular machine. Or, you can run a command remotely, e.g.

    gcloud compute ssh --command "echo 'all the way from Google'" my_google_username@my_instance
    
Or, you can scp files,  (don't forget the colon at the end to indicate remote machine!) e.g. copying into your home directory

    gcloud compute scp package_installs.sh julia_start.jl my_google_username@my_instance:

### Initializing the instance

First, edit the file `package_installs.sh` and in the git clone line put in **your** github username and password. Then run the following two commands on your local machine:

    gcloud compute scp package_installs.sh julia_start.jl my_google_username@my_instance:
    gcloud compute ssh --command "./package_installs.sh" my_google_username@my_instance
    
That will install Emacs, Julia, Git, and will then start up Julia and get the various packages that we use (ForwardDiff, HDF5, MAT, etc) and will then install our git repository in the directory `superior_colliculus_mutual_inhibition`
